In [1]:
%cd ..

d:\Blockchain\data\orai\orchai_validator


In [2]:
from labeling.tools import query, get_spark
from omegaconf import OmegaConf

## Fetch block height

In [3]:
spark = get_spark()

In [4]:
config = OmegaConf.load("config/etl.yaml")

In [5]:
df = query(spark, **config.src)

Successfully queried data from database


In [8]:
bh = df.select("block_height").groupBy("block_height").count().select("block_height")

In [9]:
bh.write.csv("data/block_height", header=True)

## Detect missing blocks

In [10]:
df = spark.read.csv("data/block_height/", header=True, inferSchema=True)

In [11]:
df.count()

2020711

In [26]:
bh = df.pandas_api()

In [29]:
bh = bh.sort_values('block_height').reset_index().to_numpy()

In [36]:
bh = bh[:, 1]

In [37]:
bh[:5]

array([7049472, 7049473, 7049474, 7049475, 7049476], dtype=int64)

In [38]:
import numpy as np

diff = np.diff(bh)

In [41]:
for i, d in enumerate(diff):
    if d != 1:
        print(bh[i], bh[i+1], "|", d)


7272731 7272972 | 241
7883969 7884210 | 241
8699979 8700001 | 22
9057005 9057016 | 11
9057065 9057076 | 11


In [10]:
from omegaconf import OmegaConf
import psycopg2
import numpy as np
import os
from collections import Counter
from argparse import ArgumentParser

def psql_connect(host, port, database, user, password, **kwargs):
    return psycopg2.connect(
        host=host, 
        port=port,
        database=database, 
        user=user, 
        password=password,
    )


config = OmegaConf.load("../config/etl_file.yaml")
cur = psql_connect(**config.src).cursor()
cur.execute(f"select distinct block_height from {config.src.table} where block_height >= 7059473 and block_height <= 9583823 order by block_height;")

In [11]:
bh = cur.fetchall()
bh = np.array([b[0] for b in bh])

In [12]:
bh[0], bh[-1]

(7059473, 9583823)

In [13]:
import numpy as np  

diff = np.diff(bh)
diff.sort()

print(diff[:10], diff[-10:])

[150 150 150 150 150 150 150 150 150 150] [150 150 150 150 150 150 150 150 150 150]


In [9]:
diff

array([  1,   1,   1, ..., 150, 150, 150])

In [20]:
bh[1421] - 7272623

0

In [19]:
3150 / 150

21.0

In [21]:
bh[1422], bh[1423], bh[1424]

(7272773, 7272923, 7273073)